<a href="https://colab.research.google.com/github/franciscodevs/Proyecto-de-Prediccion-de-Incendios/blob/main/Modelos_ML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Obtenemos la data previamente procesada

In [66]:
import io
import requests
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib.ticker import FormatStrFormatter

In [67]:
url="https://raw.githubusercontent.com/franciscodevs/Proyecto-de-Prediccion-de-Incendios/main/dataset_for_models.csv"
s=requests.get(url).content
dataset=pd.read_csv(io.StringIO(s.decode(encoding = "ISO-8859-1")))

In [68]:
dataset.head()

,superficie,fecha,lat,lng,idprovincia,causa,muertos,heridos,time_ctrl,time_ext,...,PRECIPITACION,TMIN,TMAX,VELMEDIA,RACHA,SOL,Trimestre,Año,DIR_VIENTO,PRES_RANGE
0,5.0,20010220,43.703581,-8.038777,A Coruña,1,0,0,3.55,3.55,...,0,7.0,15.6,2,16.9,10.2,1,2001,E,4.4
1,1.5,20010225,43.186836,-8.685470,A Coruña,2,0,0,2.05,2.05,...,0,6.5,11.6,3,11.1,10.2,1,2001,NE,4.7
2,1.5,20010413,43.699889,-7.984566,A Coruña,1,0,0,1.50,1.50,...,0,10.4,17.4,3,13.9,12.1,2,2001,NE,3.2
3,1.1,20010413,42.758649,-8.917814,A Coruña,3,0,0,3.10,3.10,...,0,10.4,17.4,3,13.9,12.1,2,2001,NE,3.2
4,1.5,20010413,43.063218,-9.235604,A Coruña,2,0,0,1.35,1.35,...,0,10.4,17.4,3,13.9,12.1,2,2001,NE,3.2


## MODELOS

### A continuación probaremos diferentes modelos de Machine Learning, testeando su precisión y ajuste sobre el dataset propuesto.

### KNN 

In [69]:
df = dataset.copy()

In [70]:
#Dividimos en X e y 
X = df.drop('causa', axis=1)
y = df['causa']

In [71]:
#Importamos la libreria para separar en train y test
from sklearn.model_selection import train_test_split 

#Me quedo con 30% para test y 70% para train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y) 

Fit y transformacion del X_train

In [72]:
#Comenzamos a realizar One hot encoding con las variables que no tienen un orden (como si lo tiene por ejemplo, los meses del año)
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore') #Si se agregaría otra variable seguiría funcionando
temporal = ohe.fit_transform(X_train[['idprovincia','DIR_VIENTO']]).toarray()

columns = []
for x in ohe.categories_:
    for y in x:
        columns.append(y)
_ohe = pd.DataFrame(temporal, columns=columns)
X_train.reset_index(drop=True, inplace=True)
_ohe.reset_index(drop=True, inplace=True)
X_train= pd.concat([X_train, _ohe],axis=1)
X_train.drop(['idprovincia','DIR_VIENTO'], axis=1, inplace=True)

In [73]:
from sklearn.preprocessing import StandardScaler
#Instanciamos standar scaler
se=StandardScaler()

# Escalamos los datos
X_train=pd.DataFrame(se.fit_transform(X_train),columns=X_train.columns)

In [74]:
#Instanciar PCA 
from sklearn.decomposition import PCA
pca=PCA(random_state=42,n_components=0.90)

#Transformamos el data frame
X_train_decomp=pca.fit_transform(X_train)

In [75]:
#Mostramos cantidad de columnas (19, que son las componentes que explican el 90% de variabilidad)
X_train_decomp.shape

(9083, 19)

Transformacion de los datos del X_test

In [76]:
#Se realiza One Hot Encoder
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore') #Si se agregaría otra variable seguiría funcionando
temporal = ohe.fit_transform(X_test[['idprovincia','DIR_VIENTO']]).toarray()

columns = []
for x in ohe.categories_:
    for y in x:
        columns.append(y)
_ohe = pd.DataFrame(temporal, columns=columns)
X_test.reset_index(drop=True, inplace=True)
_ohe.reset_index(drop=True, inplace=True)
X_test= pd.concat([X_test, _ohe],axis=1)
X_test.drop(['idprovincia','DIR_VIENTO'], axis=1, inplace=True)

In [77]:
from sklearn.preprocessing import StandardScaler

# Escalamos los datos
X_test=pd.DataFrame(se.transform(X_test),columns=X_test.columns)

#Transformamos el data frame
X_test_decomp=pca.transform(X_test)

#Mostramos cantidad de columnas (19, que son las componentes que explican el 90% de variabilidad)
X_test_decomp.shape

(3893, 19)

Fiteo y evaluacion

In [78]:
#Importamos el modelo
from sklearn.neighbors import KNeighborsClassifier

In [79]:
#Fiteamos
knn = KNeighborsClassifier()
knn.fit(X_train_decomp,y_train)

KNeighborsClassifier()

In [80]:
y_train_pred = knn.predict(X_train_decomp) #Prediccion en Train
y_test_pred = knn.predict(X_test_decomp) #Prediccion en Test

In [81]:
from sklearn.metrics import f1_score

#Calculo el f1 en Train
train_f1 = f1_score(y_train, y_train_pred,average='weighted')

#Calculo el f1 en Test
test_f1 = f1_score(y_test, y_test_pred,average='weighted')

print('f1 sobre el set de entrenamiento:', train_f1)
print('f1 sobre el set de evaluación:',test_f1)

knn_f1 = str(f"F1 sobre el train de KNN es: {train_f1}\nF1 sobre el test de KNN es: {test_f1}")

f1 sobre el set de entrenamiento: 0.8271174792543439
f1 sobre el set de evaluación: 0.809660972878457


In [82]:
#Creamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_test_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   3  156    1    0    0]
 [  24 3345   16    3    0]
 [   1  247    1    0    0]
 [   0   26    0    5    0]
 [   1   63    1    0    0]]


### RANDOM FOREST

In [83]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

Creacion del arbol de decision y Random Forest

In [84]:
df = dataset.copy()

In [85]:
#Separamos en X e y
#Dropeamos de x aquellas variables que no numericas y que, entiendo, no sería que el algoritmo le asigne un orden
X = df.drop(["causa","idprovincia","DIR_VIENTO"], axis=1)
y = df["causa"]

In [86]:
#Separamos en train y test!
(X_train, X_test,
y_train, y_test) = train_test_split(X,
                                    y,
                                    stratify=y,
                                    test_size=0.30,
                                    random_state=11)

In [87]:
#Creamos un arbol de decisión sencillo y lo fiteamos
tree = DecisionTreeClassifier(random_state=11)
tree.fit(X_train, y_train)

DecisionTreeClassifier(random_state=11)

In [88]:
y_train_pred = tree.predict(X_train) #Prediccion en train

In [89]:
y_test_pred = tree.predict(X_test) #Prediccion en Test

In [90]:
from sklearn.metrics import f1_score

#Calculo el f1 en Test
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

test_f1 = f1_score(y_test, y_test_pred,average='weighted')

print('f1 sobre el set de train:',train_f1)
print('f1 sobre el set de evaluación:',test_f1)

decisionTree_f1 = str(f"F1 sobre el train de Decision Tree es: {train_f1}\nF1 sobre el test de Decision Tree es: {test_f1}")

f1 sobre el set de train: 1.0
f1 sobre el set de evaluación: 0.7759502390387939


In [91]:
#Creamos un random forest!
model = RandomForestClassifier(random_state=11, n_estimators=200,
                               class_weight="balanced", max_features="log2")
model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_features='log2',
                       n_estimators=200, random_state=11)

In [92]:
y_train_pred = model.predict(X_train) #Prediccion en train

In [93]:
y_test_pred = model.predict(X_test) #Prediccion en Test

In [94]:
from sklearn.metrics import f1_score

#Calculo el f1 en Test
train_f1 = f1_score(y_train, y_train_pred, average='weighted')

test_f1 = f1_score(y_test, y_test_pred,average='weighted')

print('f1 sobre el set de train:',train_f1)
print('f1 sobre el set de evaluación:',test_f1)

randomForest_f1 = str(f"F1 sobre el train de Random Forest es: {train_f1}\nF1 sobre el test de Random Forest es: {test_f1}")

f1 sobre el set de train: 1.0
f1 sobre el set de evaluación: 0.812731691512971


### CATBOOST

In [95]:
!pip install catboost

     |████████████████████████████████| 76.1 MB 59 kB/s 


In [96]:
from catboost import CatBoostClassifier     

In [97]:
df = dataset.copy()

In [98]:
#Variables categóricas para catboost
cat_features = ['idprovincia', 'DIR_VIENTO']

In [99]:
#Instanciamos el catboost para intentar balancear el dataset
cs = CatBoostClassifier(random_state=42, eval_metric='MultiClass', class_weights={1:5,2:1,3:5,4:5,5:5}, iterations=200)

In [100]:
#Dividimos en X e y 
X = df.drop(('causa'), axis=1)
y = df[['causa']]

In [101]:
#Importamos la libreria para separar en train y test
from sklearn.model_selection import train_test_split 

In [102]:
#Me quedo con 30% para test y 70% para train
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.30, random_state=42, stratify=y) 

In [103]:
#Entrenamos el modelo
cs.fit(X_train,y_train, cat_features = ['idprovincia', 'DIR_VIENTO'])

Learning rate set to 0.345003
0:	learn: 1.3109425	total: 108ms	remaining: 21.5s
1:	learn: 1.2118288	total: 165ms	remaining: 16.4s
2:	learn: 1.1506549	total: 230ms	remaining: 15.1s
3:	learn: 1.1090990	total: 293ms	remaining: 14.4s
4:	learn: 1.0752160	total: 365ms	remaining: 14.2s
5:	learn: 1.0572939	total: 422ms	remaining: 13.6s
6:	learn: 1.0285763	total: 487ms	remaining: 13.4s
7:	learn: 1.0137742	total: 544ms	remaining: 13.1s
8:	learn: 0.9947274	total: 603ms	remaining: 12.8s
9:	learn: 0.9870330	total: 636ms	remaining: 12.1s
10:	learn: 0.9784960	total: 730ms	remaining: 12.5s
11:	learn: 0.9655101	total: 836ms	remaining: 13.1s
12:	learn: 0.9532126	total: 965ms	remaining: 13.9s
13:	learn: 0.9456803	total: 1.07s	remaining: 14.2s
14:	learn: 0.9355236	total: 1.2s	remaining: 14.8s
15:	learn: 0.9254901	total: 1.33s	remaining: 15.2s
16:	learn: 0.9202044	total: 1.44s	remaining: 15.5s
17:	learn: 0.9135933	total: 1.55s	remaining: 15.6s
18:	learn: 0.9090510	total: 1.63s	remaining: 15.6s
19:	learn: 0

In [104]:
y_train_pred = cs.predict(X_train) #Prediccion en Train
y_test_pred = cs.predict(X_test) #Prediccion en Test

In [105]:
from sklearn.metrics import f1_score

#Calculo el accuracy en Train
train_f1 = f1_score(y_train, y_train_pred,average='weighted')

#Calculo el accuracy en Test
test_f1 = f1_score(y_test, y_test_pred,average='weighted')

print('f1 sobre el set de entrenamiento:', train_f1)
print('f1 sobre el set de evaluación:',test_f1)

catBoost_f1 = str(f"F1 sobre el train de Catboost es: {train_f1}\nF1 sobre el test de Catboost es: {test_f1}")

f1 sobre el set de entrenamiento: 0.9714339224644294
f1 sobre el set de evaluación: 0.8124741194984982


In [106]:
#Creamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_test_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[  21  125   13    1    0]
 [  71 3182  112    8   15]
 [  18  199   30    0    2]
 [   0   21    0    9    1]
 [   0   63    2    0    0]]


### Support Vector Machine (Polinomial)

In [107]:
df = dataset.copy()

In [108]:
#Importamos la libreria para separar en train y test
from sklearn.model_selection import train_test_split 

#Separamos en X e y
#Dropeamos de x aquellas variables que no numericas y que, entiendo, no sería que el algoritmo le asigne un orden
X = df.drop(["causa","idprovincia","DIR_VIENTO"], axis=1)
y = df["causa"].to_numpy()

In [109]:
# Training and testing set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)
print(f"Train Data: {X_train.shape}, {y_train.shape}")
print(f"Test Data: {X_test.shape}, {y_test.shape}")

Train Data: (9083, 22), (9083,)
Test Data: (3893, 22), (3893,)


In [110]:
from sklearn import svm

#Entrenamos el SVM en un Kernel Polinomial debido a sus multiples clases en el target
modelo = svm.SVC(kernel='poly', degree=3, C=1)
modelo.fit(X_train, y_train)

y_train_pred = modelo.predict(X_train)
y_test_pred = modelo.predict(X_test)


In [111]:
from sklearn.metrics import f1_score

#Calculo el f1 en Train
train_f1 = f1_score(y_train, y_train_pred,average='weighted')

#Calculo el f1 en Test
test_f1 = f1_score(y_test, y_test_pred,average='weighted')

print('f1 sobre el set de entrenamiento:', train_f1)
print('f1 sobre el set de evaluación:',test_f1)

SVM_f1 = str(f"F1 sobre el train de Random Forest es: {train_f1}\nF1 sobre el test de Random Forest es: {test_f1}")

f1 sobre el set de entrenamiento: 0.8102719041846572
f1 sobre el set de evaluación: 0.8091849190040197


In [112]:
#Creamos la matriz de confusión
from sklearn.metrics import confusion_matrix
matriz = confusion_matrix(y_test, y_test_pred)
print('Matriz de Confusión:')
print(matriz)

Matriz de Confusión:
[[   0  165    0    0    0]
 [   0 3386    0    0    0]
 [   0  250    0    0    0]
 [   0   33    0    0    0]
 [   0   59    0    0    0]]


## Métricas

Notamos cierto desbalance en los datos por lo que nos inclinamos por no utilizar la métrica de accuracy. Además, centrándonos ya en el problema específico planteado (lograr una mejor distribución de personal y medios para reducir perdidas y costos), notamos que no sería tan grave un falso positivo o un falso negativo, aunque si necesitamos que el modelo aporte valor y sirva para tomar mejores decisiones.
De ésta manera, nos inclinamos por usar la matriz de confusión, y en particular la métrica “F1 score” que es la media armónica de precisión y exhaustividad. En otras palabras, creemos que es una métrica 
acertada para evaluar nuestro problema específico; cuánto más se acerque la métrica a 1, mejor será nuestro modelo.

In [119]:
# Resultado de KNN
print(knn_f1)

F1 sobre el train de KNN es: 0.8271174792543439
F1 sobre el test de KNN es: 0.809660972878457


In [120]:
# Resultado de Arboles de decisión
print(decisionTree_f1)

F1 sobre el train de Decision Tree es: 1.0
F1 sobre el test de Decision Tree es: 0.7759502390387939


In [121]:
# Resultado de Random Forest
print(randomForest_f1)

F1 sobre el train de Random Forest es: 1.0
F1 sobre el test de Random Forest es: 0.812731691512971


In [122]:
# Resultado de Catboost
print(catBoost_f1)

F1 sobre el train de Catboost es: 0.9714339224644294
F1 sobre el test de Catboost es: 0.8124741194984982


In [123]:
# Resultado de Support Vector Machine
print(SVM_f1)

F1 sobre el train de Random Forest es: 0.8102719041846572
F1 sobre el test de Random Forest es: 0.8091849190040197
